<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-04 14:26:46
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -4.69 K (-0.03%)
Current PnL: -13.43 L (-9.36%)
CY Booked + Current PnL: -6.69 L (-4.66%)
-------------------
Total profit:  4.53 L
Total loss:  -17.96 L
-------------------
Total Booked + Current PnL: 20.29 L (17.41%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.94%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.84 L (52.65%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-19.90,H-LC,82.81,111040.0,6728.0,10804.0,0.17,6.45,9.73,16.81,27.0,0.62,0.81,23.53,X5K,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-16.85,H-LC,86.46,113280.0,9714.0,10999.0,-0.17,9.38,9.71,20.00,30.0,0.88,0.83,24.77,X40N,BTT,HEALTHCARE
3,ADANIPORTS,1103.69,1583.00,0.58,M-LC,26.04,100919.0,22557.0,11474.0,-0.68,28.79,11.37,43.43,77.0,1.97,0.74,35.64,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,30.73,173506.0,32666.0,11833.0,0.67,23.19,6.82,31.59,89.0,2.76,1.27,50.72,XY24,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,50.53,M-SC,93.75,130950.0,-11529.0,12152.0,-1.26,-8.09,9.28,0.44,243.0,-0.95,0.96,54.56,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142827.0,985.0,55588.0,1.02,0.69,38.92,39.88,259.0,0.02,1.05,44.74,XY24,NTT,POWER
12,BANKINDIA,116.91,190.00,-31.29,M-MC,19.79,180623.0,815.0,111589.0,-0.89,0.45,61.78,62.52,171.0,0.01,1.32,34.69,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,58.33,151893.0,980.0,59466.0,0.68,0.65,39.15,40.06,102.0,0.02,1.11,24.24,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-49.40,H-MC,61.46,101198.0,694.0,29944.0,-0.68,0.69,29.59,30.49,97.0,0.02,0.74,33.32,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,12.50,49339.0,8084.0,2724.0,5.51,19.59,5.52,26.19,1.0,2.97,0.36,16.16,XY24,ATH,IT
33,GREENPANEL,375.16,537.00,218.48,M-SC,91.67,142276.0,-40802.0,119782.0,4.85,-22.29,84.19,43.14,216.0,-0.34,1.04,33.83,XY24,NTT,MISC
65,RELAXO,902.64,1176.00,-35.41,H-SC,65.62,58368.0,-57170.0,92157.0,4.20,-49.48,157.89,30.28,134.0,-0.62,0.43,14.03,X40N,NTT,FOOTWEAR
70,SATIN,199.77,274.00,-14.59,H-SC,87.50,168330.0,-27644.0,100459.0,3.08,-14.11,59.68,37.16,130.0,-0.28,1.23,25.71,XY24,NTT,FINANCE
55,LALPATHLAB,2804.19,3545.00,-32.17,H-SC,30.73,155958.0,1728.0,39021.0,2.86,1.12,25.02,26.42,125.0,0.04,1.14,22.99,X40N,NTT,HEALTHCARE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SAMMAANCAP,170.35,326.00,-163.64,M-SC,27.08,73794.0,-28416.0,121804.0,-9.39,-27.80,165.06,91.37,206.0,-0.23,0.54,19.79,XY25,NTT,FINANCE
11,BANDHANBNK,214.25,400.00,-30.37,H-SC,71.88,208046.0,-40484.0,255959.0,-2.49,-16.29,123.03,86.70,129.0,-0.16,1.53,37.63,XY24,NTT,BANKS
30,DMART,3764.92,5201.00,-16.87,X-LC,62.50,208789.0,24308.0,46059.0,-2.02,13.18,22.06,38.14,15.0,0.53,1.53,25.18,X40N,NTT,FMCG
4,ALKYLAMINE,2347.98,4546.37,6.09,X-SC,98.96,98797.0,-2166.0,96693.0,-1.86,-2.15,97.87,93.63,64.0,-0.02,0.72,47.16,SR,ATH,CHEMICALS
68,SAIL,130.64,228.00,92.75,M-MC,48.96,230869.0,5907.0,161747.0,-1.78,2.63,70.06,74.53,185.0,0.04,1.69,34.04,XY24,BTT,STEEL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,50.53,M-SC,93.75,130950.0,-11529.0,12152.0,-1.26,-8.09,9.28,0.44,243.0,-0.95,0.96,54.56,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.70,M-SC,10.42,147820.0,-23321.0,23267.0,-0.07,-13.63,15.74,-0.03,189.0,-1.00,1.08,6.24,OX40N,NTT,DURABLES
92,VIPIND,488.80,489.0,-212.91,H-SC,96.88,81412.0,-13904.0,13946.0,1.08,-14.59,17.13,0.04,154.0,-1.00,0.60,60.58,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,126.56,H-SC,90.62,148701.0,-25858.0,25889.0,-0.17,-14.81,17.41,0.02,127.0,-1.00,1.09,28.95,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.31,X-MC,17.71,86870.0,-15782.0,15776.0,-0.16,-15.37,18.16,-0.00,58.0,-1.00,0.64,32.05,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,12.50,49339.0,8084.0,2724.0,5.51,19.59,5.52,26.19,1.0,2.97,0.36,16.16,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.38,X-LC,41.67,266393.0,-29541.0,123447.0,0.43,-9.98,46.34,31.73,1.0,-0.24,1.95,6.63,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,68.75,292468.0,30867.0,114764.0,0.94,11.80,39.24,55.67,2.0,0.27,2.14,18.62,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-16.55,X-LC,32.29,261576.0,-8771.0,43395.0,0.96,-3.24,16.59,12.81,6.0,-0.20,1.92,9.40,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-12.22,X-LC,28.12,278390.0,12964.0,43958.0,0.04,4.88,15.79,21.44,9.0,0.29,2.04,11.85,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,0.15,M-LC,30.73,173506.0,32666.0,11833.0,0.67,23.19,6.82,31.59,89.0,2.76,1.27,50.72,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,0.58,M-LC,26.04,100919.0,22557.0,11474.0,-0.68,28.79,11.37,43.43,77.0,1.97,0.74,35.64,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,69.79,72390.0,37941.0,2005.0,-0.91,110.14,2.77,115.97,NaN,18.92,0.53,10.70,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,48.96,230869.0,5907.0,161747.0,-1.78,2.63,70.06,74.53,185.0,0.04,1.69,34.04,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,27.50,H-SC,51.04,130347.0,5050.0,54068.0,0.78,4.03,41.48,47.18,139.0,0.09,0.96,63.07,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.54,H-SC,38.54,117320.0,15316.0,29705.0,-1.61,15.02,25.32,44.14,124.0,0.52,0.86,33.69,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,0.58,M-LC,26.04,100919.0,22557.0,11474.0,-0.68,28.79,11.37,43.43,77.0,1.97,0.74,35.64,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,30.73,173506.0,32666.0,11833.0,0.67,23.19,6.82,31.59,89.0,2.76,1.27,50.72,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,69.79,72390.0,37941.0,2005.0,-0.91,110.14,2.77,115.97,NaN,18.92,0.53,10.70,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.56,M-SC,85.42,202537.0,30916.0,131163.0,-1.70,18.01,64.76,94.43,198.0,0.24,1.48,49.48,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,84.38,167214.0,16269.0,93188.0,0.99,10.78,55.73,72.51,69.0,0.17,1.23,15.18,XR,NTT,IT
54,KPIGREEN,497.21,731.79,27.50,H-SC,51.04,130347.0,5050.0,54068.0,0.78,4.03,41.48,47.18,139.0,0.09,0.96,63.07,MH,ATH,POWER
45,INDIAMART,2327.09,4850.92,-48.26,H-SC,80.21,134678.0,11342.0,122422.0,-1.51,9.20,90.90,108.45,122.0,0.09,0.99,33.60,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,7.29,17584.0,-32009.0,934.0,1.92,-64.54,5.31,-62.66,NaN,-34.27,0.13,17.92,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,67104.0,-46445.0,86497.0,-0.37,-40.90,128.90,35.27,261.0,-0.54,0.49,64.24,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.00,7183.33,M-SC,88.54,79927.0,-13883.0,90501.0,0.66,-14.80,113.23,81.67,242.0,-0.15,0.59,54.78,XR,NTT,CERAMICS
58,MASFIN,326.60,399.50,-18.25,H-SC,79.17,92790.0,-5190.0,27058.0,0.62,-5.30,29.16,22.32,137.0,-0.19,0.68,34.58,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.09,X-SC,98.96,98797.0,-2166.0,96693.0,-1.86,-2.15,97.87,93.63,64.0,-0.02,0.72,47.16,SR,ATH,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142827.0,985.0,55588.0,1.02,0.69,38.92,39.88,259.0,0.02,1.05,44.74,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,6.09,X-SC,98.96,98797.0,-2166.0,96693.0,-1.86,-2.15,97.87,93.63,64.0,-0.02,0.72,47.16,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-2.82,H-SC,97.92,182190.0,17520.0,51560.0,-1.11,10.64,28.30,41.95,121.0,0.34,1.34,18.90,X40N,ATH,AC
92,VIPIND,488.80,489.00,-212.91,H-SC,96.88,81412.0,-13904.0,13946.0,1.08,-14.59,17.13,0.04,154.0,-1.00,0.60,60.58,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,67104.0,-46445.0,86497.0,-0.37,-40.90,128.90,35.27,261.0,-0.54,0.49,64.24,XR,NTT,HOTELS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.89
1,25,41.83
2,50,70.41


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.44
LC    33.91
MC    21.56
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.51
X40      12.27
XY25     11.93
X40N     11.92
XR       10.13
OX40N     8.22
AR        8.18
MH        1.70
X5K       1.46
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.67
M-SC    15.94
H-LC    14.11
X-LC    12.12
H-MC     9.53
X-MC     7.61
M-LC     6.64
X-SC     4.38
M-MC     4.06
L-SC     2.45
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.06,-1.88,35.98
IT,12.14,-9.68,63.54
FINANCE,7.84,-12.67,53.37
BANKS,7.83,-6.75,50.30
MISC,6.65,-14.73,67.84
PAINTS,5.73,-11.71,33.73
ELECTRICAL,5.49,1.88,43.95
HEALTHCARE,4.57,-3.12,33.24
AUTO,4.33,-14.22,64.44


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2701248.0,27
XR,1076618.0,13
AR,1026348.0,10
X40,644256.0,9
X40N,537698.0,11
XY25,490281.0,8
OX40N,308642.0,11
SR,178063.0,2
MH,84012.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1965850.0,20
M-SC,1763311.0,21
H-LC,613801.0,12
H-MC,532419.0,8
X-LC,520177.0,7
M-MC,415569.0,3
X-MC,414123.0,6
X-SC,320085.0,5
M-LC,268106.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      875374.0      6
M-SC       XY24      807758.0      7
H-SC       AR        523778.0      3
M-SC       XR        335176.0      4
M-MC       XY24      303980.0      2
H-MC       XY24      297455.0      3
X-LC       X40       238211.0      2
H-SC       X40N      212443.0      4
           XR        211000.0      3
H-LC       X40N      196051.0      4
M-SC       AR        182626.0      2
           OX40N     163464.0      5
M-LC       XY24      160297.0      3
H-LC       X40       155501.0      3
L-SC       XR        148300.0      2
X-MC       X40       148020.0      2
X-SC       XR        123472.0      1
M-SC       XY25      121804.0      1
X-LC       XY25      118648.0      3
           AR        117259.0      1
M-MC       XR        111589.0      1
X-MC       XY25      103749.0      1
X-SC       SR         96693.0      1
X-MC       XY24       93552.0      1
M-LC       XR         93188.0      1
H-SC       OX40N      89187.0      3
H-MC       XY25       84024.0      1
M-SC       SR         81370.0      1
H-LC       AR         81085.0      1
M-SC       X40        71113.0      1
H-LC       X200       69098.0      1
           X5K        68185.0      2
L-SC       AR         60129.0      1
X-SC       XY24       59705.0      1
H-MC       AR         59466.0      1
L-SC       XY24       55588.0      1
H-SC       MH         54068.0      1
L-MC       XR         53893.0      1
X-MC       X40N       53026.0      1
L-LC       XY25       47435.0      1
X-LC       X40N       46059.0      1
H-LC       XY24       43881.0      1
X-SC       OX40N      40215.0      2
H-MC       X40        31411.0      1
           X40N       30119.0      1
           MH         29944.0      1
X-MC       OX40N      15776.0      1
M-LC       XY25       14621.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
